In [93]:
import pandas as pd
import glob
import os

csvFile = './STED'
outFile = './Test.csv'

fileList = glob.glob(os.path.join(csvFile, '*.csv'))
print(fileList)

allData = []

for file in fileList:
    df = pd.read_csv(file)
    allData.append(df)

allCsv = pd.concat(allData, axis=0, ignore_index=False)
allCsv.to_csv(outFile, index=False)

['./STED/chunk6.csv', './STED/chunk5.csv', './STED/chunk2.csv', './STED/chunk4.csv', './STED/chunk3.csv', './STED/chunk1.csv']


In [98]:
import pandas as pd
data = pd.read_csv('./Test.csv')
print(data)

        network_code receiver_code receiver_type  receiver_latitude  \
0                 NN           PAH            HH          39.710602   
1                 NN           PAH            HH          39.710602   
2                 NN           PAH            HH          39.710602   
3                 NN           PAH            HH          39.710602   
4                 NN           PAH            HH          39.710602   
...              ...           ...           ...                ...   
1265652           7F          X201            HH          35.292599   
1265653           7F          X201            HH          35.292599   
1265654           7F          X201            HH          35.292599   
1265655           7F          X201            HH          35.292599   
1265656           7F          X201            HH          35.292599   

         receiver_longitude  receiver_elevation_m  p_arrival_sample p_status  \
0               -119.385399                1520.0             600.0

In [120]:
import pandas as pd
import h5py
import numpy as np
import matplotlib.pyplot as plt

file_name = "./STED/chunk1.hdf5"
csv_file = "./Test.csv"

df = pd.read_csv(csv_file)
df.info()

/root/anaconda3/envs/tensorflow-2.1.0/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (7,11,13,14,15,18,19,20,21,22,24,25,26,30,31) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1265657 entries, 0 to 1265656
Data columns (total 35 columns):
 #   Column                            Non-Null Count    Dtype  
---  ------                            --------------    -----  
 0   network_code                      1265613 non-null  object 
 1   receiver_code                     1265657 non-null  object 
 2   receiver_type                     1265657 non-null  object 
 3   receiver_latitude                 1265657 non-null  float64
 4   receiver_longitude                1265657 non-null  float64
 5   receiver_elevation_m              1265657 non-null  float64
 6   p_arrival_sample                  1030231 non-null  float64
 7   p_status                          1030231 non-null  object 
 8   p_weight                          1030057 non-null  float64
 9   p_travel_sec                      1030231 non-null  float64
 10  s_arrival_sample                  1030231 non-null  float64
 11  s_status                          103

In [165]:
import pandas as pd
import h5py
import numpy as np
import matplotlib.pyplot as plt

file_name = "./STED/chunk1.hdf5"
csv_file = "./Test.csv"
df = pd.read_csv(csv_file)
print(f'total events in csv file: {len(df)}')

df = df[(df.trace_category == 'earthquake_local') & (df.source_magnitude <= 2) | (df.trace_category == 'noise')]
print(f'total events selected: {len(df)}')
df.to_csv("./Test_3.csv")

ev_list = df['trace_name'].to_list()

dtfl = h5py.File(file_name, 'r')
processed = h5py.File("Test_3.hdf5", 'w')
grp = processed.create_group("data")

for c, evi in enumerate(ev_list):
    dataset = dtfl.get('data/'+str(evi)) 
    CurData = np.array(dataset)
    grp.create_dataset(str(evi), data=CurData)
    if c % 1000 == 0:
        print(str(c) + " / " + str(len(ev_list)))

dtfl.close()
processed.close()

total events in csv file: 1265657
total events selected: 1008704


TypeError: Object dtype dtype('O') has no native HDF5 equivalent

In [164]:
import pandas as pd
import h5py
import numpy as np
import matplotlib.pyplot as plt

file_name = "./STED/chunk1.hdf5"
csv_file = "./Test.csv"
df = pd.read_csv(csv_file)
print(f'total events in csv file: {len(df)}')

df = df[(df.trace_category == 'earthquake_local') & (df.source_magnitude <= 2) | (df.trace_category == 'noise')]
print(f'total events selected: {len(df)}')
df.to_csv("./Test_1.csv")

ev_list = df['trace_name'].to_list()

dtfl = h5py.File(file_name, 'r')
processed = h5py.File("Test_1.hdf5", 'w')
grp = processed.create_group("data")

for c, evi in enumerate(ev_list):
    dataset = dtfl.get('data/'+str(evi)) 
    CurData = np.array(dataset)
    grp.create_dataset(str(evi), data='string')
    if c % 1000 == 0:
        print(str(c) + " / " + str(len(ev_list)))

dtfl.close()
processed.close()

total events in csv file: 1265657
total events selected: 1008704
0 / 1008704
1000 / 1008704
2000 / 1008704
3000 / 1008704
4000 / 1008704
5000 / 1008704
6000 / 1008704
7000 / 1008704
8000 / 1008704
9000 / 1008704
10000 / 1008704
11000 / 1008704
12000 / 1008704
13000 / 1008704
14000 / 1008704
15000 / 1008704
16000 / 1008704
17000 / 1008704
18000 / 1008704
19000 / 1008704
20000 / 1008704
21000 / 1008704
22000 / 1008704
23000 / 1008704
24000 / 1008704
25000 / 1008704
26000 / 1008704
27000 / 1008704
28000 / 1008704
29000 / 1008704
30000 / 1008704
31000 / 1008704
32000 / 1008704
33000 / 1008704
34000 / 1008704
35000 / 1008704
36000 / 1008704
37000 / 1008704
38000 / 1008704
39000 / 1008704
40000 / 1008704
41000 / 1008704
42000 / 1008704
43000 / 1008704
44000 / 1008704
45000 / 1008704
46000 / 1008704
47000 / 1008704
48000 / 1008704
49000 / 1008704
50000 / 1008704
51000 / 1008704
52000 / 1008704
53000 / 1008704
54000 / 1008704
55000 / 1008704
56000 / 1008704
57000 / 1008704
58000 / 1008704
5900

485000 / 1008704
486000 / 1008704
487000 / 1008704
488000 / 1008704
489000 / 1008704
490000 / 1008704
491000 / 1008704
492000 / 1008704
493000 / 1008704
494000 / 1008704
495000 / 1008704
496000 / 1008704
497000 / 1008704
498000 / 1008704
499000 / 1008704
500000 / 1008704
501000 / 1008704
502000 / 1008704
503000 / 1008704
504000 / 1008704
505000 / 1008704
506000 / 1008704
507000 / 1008704
508000 / 1008704
509000 / 1008704
510000 / 1008704
511000 / 1008704
512000 / 1008704
513000 / 1008704
514000 / 1008704
515000 / 1008704
516000 / 1008704
517000 / 1008704
518000 / 1008704
519000 / 1008704
520000 / 1008704
521000 / 1008704
522000 / 1008704
523000 / 1008704
524000 / 1008704
525000 / 1008704
526000 / 1008704
527000 / 1008704
528000 / 1008704
529000 / 1008704
530000 / 1008704
531000 / 1008704
532000 / 1008704
533000 / 1008704
534000 / 1008704
535000 / 1008704
536000 / 1008704
537000 / 1008704
538000 / 1008704
539000 / 1008704
540000 / 1008704
541000 / 1008704
542000 / 1008704
543000 / 10087

967000 / 1008704
968000 / 1008704
969000 / 1008704
970000 / 1008704
971000 / 1008704
972000 / 1008704
973000 / 1008704
974000 / 1008704
975000 / 1008704
976000 / 1008704
977000 / 1008704
978000 / 1008704
979000 / 1008704
980000 / 1008704
981000 / 1008704
982000 / 1008704
983000 / 1008704
984000 / 1008704
985000 / 1008704
986000 / 1008704
987000 / 1008704
988000 / 1008704
989000 / 1008704
990000 / 1008704
991000 / 1008704
992000 / 1008704
993000 / 1008704
994000 / 1008704
995000 / 1008704
996000 / 1008704
997000 / 1008704
998000 / 1008704
999000 / 1008704
1000000 / 1008704
1001000 / 1008704
1002000 / 1008704
1003000 / 1008704
1004000 / 1008704
1005000 / 1008704
1006000 / 1008704
1007000 / 1008704
1008000 / 1008704
